### Multi-lingual model based on Transformer for Sentiment Analysis

* Conda env used: **Natural Language Processing for CPU Python 3.7**
* the model is based on a pre-trained transformer available from **Hugging Face Hub**: "nlptown/bert-base-multilingual-uncased-sentiment"
* if a GPU is available, the model does inference on GPU (faster)

**Sentiment Analysis**: we want to analyze a text and establish if the sentiment expressed is positive, negative or neutral.

In the case of the transformer used here, it is used a scale with a **number of stars** ranging from **1** (very negative) to **5** (highly positive)

In [1]:
import torch
from torch import nn
import numpy as np

# HuggingFace transformers (availale in OCI DS conda nlp env)
# see: https://github.com/huggingface/transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# from my Python file
from sentiment_analyzers import MultiLanguageSentimentAnalyzer

In [2]:
%%time

# loading the model: pass the HF model name
MODEL_NAME = "nlptown/bert-base-multilingual-uncased-sentiment"

# labels are dependent on the used model , see HF documentation
sent_analyzer = MultiLanguageSentimentAnalyzer(
    MODEL_NAME, labels=["1 star", "2 star", "3 star", "4 star", "5 star"]
)

Loading model...
Model loading completed!
CPU times: user 5.36 s, sys: 1.56 s, total: 6.93 s
Wall time: 11.7 s


### Some tests

In [3]:
# is using GPU?
sent_analyzer.get_device()

device(type='cuda')

In [4]:
%%time

sentence = "Il servizio offerto alla clientela è ottimo"
scores = sent_analyzer.score(sentence)

print(sentence)
print(sent_analyzer.format_scores(scores))
print()

Il servizio offerto alla clientela è ottimo
1 star: 0.0056|2 star: 0.0059|3 star: 0.0518|4 star: 0.323|5 star: 0.6137|

CPU times: user 144 ms, sys: 26.6 ms, total: 170 ms
Wall time: 166 ms


In [5]:
sentence = "The service you have offered to your customers is really good"

scores = sent_analyzer.score(sentence)

print(sentence)
print(sent_analyzer.format_scores(scores))
print()

The service you have offered to your customers is really good
1 star: 0.0031|2 star: 0.0043|3 star: 0.0572|4 star: 0.4137|5 star: 0.5217|



In [16]:
# sentences for tests
input_sentences = [
    "Noi pensiamo che questa riunione sia stata proficua",
    "Noi pensiamo che questa riunione sia stata molto proficua",
    "The service offered to your customers is really good",
    "E' un prodotto pessimo",
    "La sua organizzazione ha fornito un buon servizio alla clientela",
    "La sua organizzazione non ha fornito un buon servizio alla clientela",
    "Non credo che la sua organizzazione abbia fornito un buon servizio alla clientela",
    "Il prodotto non funziona, non comprero' più nulla dalla vostra azienda",
    "Io penso che la sua organizzazione non abbia fornito un buon servizio alla clientela",
    "La gestione da parte della Regione Lazio della complessa macchina dei vaccini è stata buona?",
    "La gestione da parte della Regione Lazio della complessa macchina dei vaccini è stata buona",
    "La vostra organizzazione offre servizi pessimi",
    "La vostra organizzazione offre servizi non adeguati",
    "Sono molto soddisfatto del tuo lavoro",
    "non sono del tutto sicuro che il lavoro sia adeguato",
    "l'azienda dovrebbe offrire servizi migliori",
    "il supporto offerto dal customer care non è stato adeguato",
    "il risultato è pessimo",
    "il Napoli ha giocato una partita decente",
    "il lavoro dell'allenatore è stato modesto",
]

In [12]:
%%time

# here we do inference one sentence a time (SLOWER)

# object already instantiated

for i, sentence in enumerate(input_sentences):

    #
    # here I do the scoring on a single sentence
    #
    scores = sent_analyzer.score(sentence)

    # scores is a dict of []

    print(f"{i+1}. {sentence}")
    print(sent_analyzer.format_scores(scores))
    print()

# formatting
print()

1. Noi pensiamo che questa riunione sia stata proficua
1 star: 0.0614|2 star: 0.0941|3 star: 0.3085|4 star: 0.2887|5 star: 0.2473|

2. The service offered to your customers is really good
1 star: 0.004|2 star: 0.0051|3 star: 0.0578|4 star: 0.4196|5 star: 0.5135|

3. E' un prodotto pessimo
1 star: 0.9365|2 star: 0.0558|3 star: 0.0063|4 star: 0.0008|5 star: 0.0006|

4. La sua organizzazione ha fornito un buon servizio alla clientela
1 star: 0.0041|2 star: 0.0072|3 star: 0.1284|4 star: 0.5368|5 star: 0.3235|

5. La sua organizzazione non ha fornito un buon servizio alla clientela
1 star: 0.2703|2 star: 0.4699|3 star: 0.2328|4 star: 0.0228|5 star: 0.0042|

6. Non credo che la sua organizzazione abbia fornito un buon servizio alla clientela
1 star: 0.0336|2 star: 0.0579|3 star: 0.2817|4 star: 0.4109|5 star: 0.2159|

7. Il prodotto non funziona, non comprero' più nulla dalla vostra azienda
1 star: 0.9054|2 star: 0.0825|3 star: 0.0105|4 star: 0.0009|5 star: 0.0006|

8. Io penso che la sua org

### Test batch scoring

In [17]:
%%time

# here we do batch scoring (faster)
scores = sent_analyzer.batch_score(input_sentences)

# Instead of a tensor I want the numpy vector
stars = np.argmax(scores.numpy(), axis=-1)

for i, (sentence, star) in enumerate(zip(input_sentences, stars)):
    print(f"{i+1}. {sentence}. Stelle: {star+1}")
    
# for formatting only
print()

1. Noi pensiamo che questa riunione sia stata proficua. Stelle: 3
2. Noi pensiamo che questa riunione sia stata molto proficua. Stelle: 5
3. The service offered to your customers is really good. Stelle: 5
4. E' un prodotto pessimo. Stelle: 1
5. La sua organizzazione ha fornito un buon servizio alla clientela. Stelle: 4
6. La sua organizzazione non ha fornito un buon servizio alla clientela. Stelle: 2
7. Non credo che la sua organizzazione abbia fornito un buon servizio alla clientela. Stelle: 4
8. Il prodotto non funziona, non comprero' più nulla dalla vostra azienda. Stelle: 1
9. Io penso che la sua organizzazione non abbia fornito un buon servizio alla clientela. Stelle: 2
10. La gestione da parte della Regione Lazio della complessa macchina dei vaccini è stata buona?. Stelle: 3
11. La gestione da parte della Regione Lazio della complessa macchina dei vaccini è stata buona. Stelle: 4
12. La vostra organizzazione offre servizi pessimi. Stelle: 1
13. La vostra organizzazione offre serv

### custom test

In [8]:
sentence = "E' un prodotto pessimo"

print(sentence)
print(sent_analyzer.score(sentence))
print()

E' un prodotto pessimo
[{'label': '1 star', 'score': 0.9365}, {'label': '2 star', 'score': 0.0558}, {'label': '3 star', 'score': 0.0063}, {'label': '4 star', 'score': 0.0008}, {'label': '5 star', 'score': 0.0006}]

